## TOOL UPDATED: June 20 2023, Henrik Loecke

##Double click this cell to see full description.

##You must restart the kernel after updating System_Assessment_Variables!

<!-- 

To run this notebook, click menu Cell -> Run All

User input has been moved away from this notebook so it can easily be replaced by new versions.
 
Please open System_Assessment_Variables, in the same folder as this notebook, to edit user input there.

All variables with path must start with 'r', e.g. r'C:\Projects'

It must contain the following variables:

model_area:                          Short area name like 'VSA' or LISA'  
result_specs_csv:                    CSV file linking network and runoff result file. Only needed if runoff imported.
ps_specs_csv:                        CSV file containing PS firm and station capacities.
summation_csv:                       Summarizes which elements are to be summed up, mostly used for outfalls.
summation_ps_csv:                    Summarizes which elements are to be summed up for PS assessment.
outfall_csv:                         Summarizes all outfalls that are to have overflows displayed on the map.    
result_folder:                       Folder path of result files.
map_folder:                          Folder path where maps are generated and where this tool returns output.
set_pipe_volume_0_when_max_pipe_flow_less_than_cms: 
                                     If the maximum absolute (allow reverse flow) is below this, set to 0.
 -->


In [1]:
#PERMANENT CELL 1

import os
import re
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
from mikeio.dfs0 import Dfs0
import numpy as np
import pandas as pd
import datetime as dt
import traceback
import ctypes
MessageBox = ctypes.windll.user32.MessageBoxA
from System_Assessment_Variables import *

In [2]:
#PERMANENT CELL 2
#List all network elements to be imported.

try:

    flow_pipes = set()

    #Add outfalls
    outfalls = pd.read_csv(outfall_csv,dtype={'Weir': str,'Outfall': str})
    outfalls['Res_ID'] = ''
    for index, row in outfalls.iterrows():
        prefix = ''

        if row['Layer'] == 'Summation':
            prefix = ''
        elif row['Layer'] != 'msm_Link':
            prefix = row['Layer'][4:] + ':'
        muid = prefix + row['Weir']

        if row['Layer'] != 'Summation':
            outfalls.iloc[index,3] = row['Layer'][4:]
        outfalls.iloc[index,4] = muid


        if row['Layer'] != 'Summation':
            flow_pipes.add(muid)

    catchments = set()
    summation_df = pd.read_csv(summation_csv,dtype={'MUID': str,'SUMTO': str})
    summation_df_ps = pd.read_csv(summation_ps_csv,dtype={'MUID': str,'SUMTO': str})
    summation_df = pd.concat([summation_df,summation_df_ps])
    summation_df.reset_index(drop=True,inplace=True)
    summation_df['TS_ID'] = ''

    for index, row in summation_df.iterrows():
        prefix = ''

        if row['Layer'] != 'msm_Link' and row['Layer'] != 'ms_Catchment':
            prefix = row['Layer'][row['Layer'].find('_') + 1:] + ':'

        muid = prefix + row['MUID']
        summation_df.iloc[index,3] = muid

        summation_df.iloc[index,0] = muid

        if '-Negative' in muid:
            muid = muid[:-9]

        if row['Layer'] == 'ms_Catchment':
            catchments.add(muid)
        else:
            flow_pipes.add(muid)
        
except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 2', b'Error', 0)
    raise ValueError("Error")
        

        

In [3]:
#PERMANENT CELL 3
#Import network (and runoff if applicable) result files

pipes_set_to_zero = set()

try:
    ps_specs = pd.read_csv(ps_specs_csv)
    ps_specs.set_index('PS',inplace=True)

    first_runoff = True
    if len(catchments) > 0:
        result_specs = pd.read_csv(result_specs_csv)
        for f in result_specs.Runoff.unique():

                print('Importing ' + f + " at " + str(dt.datetime.now()))
                res1d = Res1D(result_folder + '\\' + f)

                for catchment in catchments:

                    catchment_df = pd.DataFrame(index = res1d.time_index)
                    catchment_df['ResultFile'] = f
                    catchment_df['MUID'] = catchment
                    catchment_df['DateTimeRef'] = catchment_df.index
                    ts = res1d.query.GetCatchmentValues(catchment, "TotalRunOff")
                    if ts == None:
                        raise ValueError("Catchment '" + catchment + "' not found in " + f)
                    catchment_df['Discharge'] = ts
                    catchment_df['Discharge'] = catchment_df['Discharge']*1000

                    catchment_df['Seconds'] = catchment_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                    catchment_df['Volume'] = catchment_df.Discharge * catchment_df.Seconds / 1000
                    catchment_df.drop(columns=['Seconds'],inplace=True)


                    if first_runoff == True:
                        catchment_df_all = catchment_df.copy()
                    else:
                        catchment_df_all = pd.concat([catchment_df_all,catchment_df])
                    first_runoff = False

        result_specs.rename(columns={'Runoff':'ResultFile'},inplace=True)
        catchment_df_all = pd.merge(catchment_df_all,result_specs,how='left',on=['ResultFile'])
        catchment_df_all.drop(columns='ResultFile',inplace=True)
        catchment_df_all.rename(columns={'Network':'ResultFile'},inplace=True)
        catchment_df_all = catchment_df_all[['ResultFile', 'MUID', 'DateTimeRef', 'Discharge', 'Volume']]

    results = []
    first_level = True
    first_flow = True
    first_velocity = True

    res_stats = []
    for i in range(20):
        res_stats.append(['FORCETEXT','FORCETEXT','FORCETEXT',0])

    network_result_files = []
    
    if len(catchments) > 0:    
        network_result_files = list(result_specs.Network.unique())
    else:   
        for f in os.listdir(result_folder):
            if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f and not 'hotstart' in f.lower() and not 'runoff' in f.lower() and not 'rdii' in f.lower(): 
                network_result_files.append(f)
    
    for f in network_result_files:

        res1d = Res1D(result_folder + '\\' + f)
        timestep_seconds = (max(res1d.time_index) - min(res1d.time_index)).total_seconds() / (len(res1d.time_index)-1)
        reaches = res1d.data.Reaches
        nodes = res1d.data.Nodes

        print ("Importing network " + f + " at " + str(dt.datetime.now()))

        first_round = True

        for i, reach in enumerate(reaches):

            muid = reach.Id[:reach.Id.rfind('-')]

            max_flow = max(res1d.query.GetReachEndValues(muid, 'Discharge'))
            min_flow = min(res1d.query.GetReachEndValues(muid, 'Discharge'))
            max_abs_flow = max(abs(max_flow),abs(min_flow))
            if max_abs_flow < set_pipe_volume_0_when_max_pipe_flow_less_than_cms and \
                                                    not( muid.startswith('Orifice:') or \
                                                        muid.startswith('Pump:') or \
                                                        muid.startswith('Weir:') or muid.startswith('Valve:') ):
                volume = 0
            else:
                volume = sum(res1d.query.GetReachEndValues(muid, 'Discharge')) * timestep_seconds

            res_stats.append([f,muid,'MAXFLOW',max_flow])
            res_stats.append([f,muid,'VOLUME',volume])
            try:
                velocity = max(res1d.query.GetReachEndValues(muid, "FlowVelocity"))
                res_stats.append([f,muid,'MAXVELOCITY',velocity])
            except:
                pass #only pipes have velocity

            if muid in flow_pipes or muid in list(summation_df.MUID):

                values = res1d.query.GetReachEndValues(muid, 'Discharge')
                flow_df = pd.DataFrame(index = res1d.time_index)
                flow_df['ResultFile'] = f
                flow_df['MUID'] = muid
                flow_df['DateTimeRef'] = flow_df.index
                if max_abs_flow < set_pipe_volume_0_when_max_pipe_flow_less_than_cms and \
                                                    not( muid.startswith('Orifice:') or \
                                                        muid.startswith('Pump:') or \
                                                        muid.startswith('Weir:') or muid.startswith('Valve:') ):
                    flow_df['Discharge'] = 0
                    pipes_set_to_zero.add(muid)
                else:
                    flow_df['Discharge'] = values  
                    
                flow_df['Discharge'] = flow_df['Discharge'] * 1000

                flow_df['Seconds'] = flow_df.index.to_series().diff().astype('timedelta64[s]').fillna(method='bfill')
                flow_df['Volume'] = flow_df.Discharge * flow_df.Seconds / 1000
                flow_df.drop(columns=['Seconds'],inplace=True)

                if first_flow == True:
                    flow_df_all = flow_df.copy()
                else:
                    flow_df_all = pd.concat([flow_df_all,flow_df])
                first_flow = False

        first_round = False

        for node in nodes:
            muid = node.Id
            max_level = max(res1d.query.GetNodeValues(muid, "WaterLevel"))
            res_stats.append([f,muid,'MAXLEVEL',max_level])

        flood_types = ['WaterFlowRateAboveGround', 'WaterSpillDischarge']
        
        flooding_result_available = False
        if os.path.exists(result_folder + '\\' + f[:-6] + '.ADDOUT.res1d'):
            print ("Importing spilling " + f[:-6] + ".ADDOUT.res1d at " + str(dt.datetime.now()))
            res1d = Res1D(result_folder + '\\' + f[:-6] + '.ADDOUT.res1d')
            flooding_result_available = True
        elif flood_types[0] in res1d.quantities or flood_types[1] in res1d.quantities:
            pass #Maintain same res1d file which has flooding (MIKE+)    
            flooding_result_available = True
        else:
            print('WARNING! ' + f[:-6] + '.ADDOUT.res1d does not exist and flooding not available in ' + f + '.')
        
        if flooding_result_available == True:         
            
            df = pd.DataFrame(index=res1d.time_index)

            for node in res1d.data.Nodes:
                muid = node.Id
                for i, flood_type in enumerate(flood_types):
                    flood_rate = res1d.query.GetNodeValues(muid,flood_type)
                    if flood_rate != None:
                        if max(flood_rate) > 0:
                            res_stats.append([f,muid,'MAXSPILLRATE',max(flood_rate)])
                            if i == 0: #Normal
                                spill_volume = max(res1d.query.GetNodeValues(muid,'WaterVolumeAboveGround'))
                            else:
                                spill_volume =  sum(flood_rate) * timestep_seconds
                            res_stats.append([f,muid,'SPILLVOLUME',spill_volume])

    if len(catchments) > 0: 
        catchment_df_all_filter  = catchment_df_all.copy()
        times = list(flow_df_all.DateTimeRef.unique())
        catchment_df_all_filter.set_index(catchment_df_all_filter.DateTimeRef,inplace=True)
        catchment_df_all_filter =  catchment_df_all_filter[catchment_df_all_filter['DateTimeRef'].isin(times)]

    negatives = list(summation_df.query("MUID.str.endswith('-Negative')").MUID.unique())
    original_negatives = []
    for negative in negatives:
        original_negatives.append(negative[:-9])
    negatives_df = flow_df_all[flow_df_all.MUID.isin(original_negatives)].copy()
    negatives_df.MUID = negatives_df.MUID + '-Negative'
    negatives_df.Discharge = negatives_df.Discharge * -1
    negatives_df.Volume = negatives_df.Volume * -1

    flow_df_all = pd.concat([flow_df_all,negatives_df])

    if len(catchments) > 0: 
        flow_df_all = pd.concat([flow_df_all,catchment_df_all_filter])

    #Summation
    print ("Doing summation at " + str(dt.datetime.now()))             

    df_result_sum = pd.merge(flow_df_all,summation_df,how='inner',on=['MUID'])
    df_result_sum = df_result_sum.groupby(['ResultFile','SUMTO','DateTimeRef']).agg({'Discharge':'sum','Volume':'sum'})
    df_result_sum.reset_index(inplace=True)
    df_result_sum.set_index('DateTimeRef',drop=False,inplace=True)
    df_result_sum.rename(columns = {'SUMTO':'MUID'},inplace=True)

    for index, row in df_result_sum[['ResultFile','MUID','Discharge']].groupby(['ResultFile','MUID']).max().reset_index().iterrows():
        res_stats.append([row['ResultFile'],row['MUID'],'MAXFLOW',row['Discharge']/1000])

    for index, row in df_result_sum[['ResultFile','MUID','Volume']].groupby(['ResultFile','MUID']).sum().reset_index().iterrows():
        res_stats.append([row['ResultFile'],row['MUID'],'VOLUME',row['Volume']])

    flow_df_all = pd.concat([flow_df_all,df_result_sum])

    not_founds = []
    for muid in summation_df.TS_ID.unique():
        if not muid in flow_df_all.MUID.unique():
            not_founds.append(muid)

    if len(not_founds) > 0:
        error_message = 'The following elements in summation_df were not found: \n' 
        error_message += ','.join(not_founds) + '.\n'
        raise ValueError(error_message)

    res_stats_df = pd.DataFrame(res_stats,columns=['RESULT','MUID','TYPE','READING'])

    res_stats_df.to_csv(map_folder + '\\Res_Stats.csv',index=False)

    ps_stats = []

    for result in network_result_files:
        for ps in summation_df_ps.SUMTO.unique():
            if ps[-7:] != 'Outflow':
                print (ps)
                inflow = max(flow_df_all.query("ResultFile == '" + result + "' & MUID == '" + ps + "'").Discharge)
                outflow = max(flow_df_all.query("ResultFile == '" + result + "' & MUID == '" + ps + ' Outflow' + "'").Discharge)
                firm_capacity = ps_specs.loc[ps,'Firm Capacity']
                station_capacity = ps_specs.loc[ps,'Wet Weather Capacity']
                if inflow >= station_capacity:
                    color = "BLACK"
                elif inflow >= firm_capacity:
                    color = "RED"
                else:
                    color = "GREEN"
                ps_stats.append([ps,result,inflow,outflow,firm_capacity,station_capacity,color])
    ps_stats_df = pd.DataFrame(ps_stats,columns = ['PS','ResultFile','MaxTotalInflow','MaxTotalOutflow', 'FirmCapacity', 'StationCapacity', 'Color'])
    ps_stats_df.to_csv(map_folder + '\\PS_Stats.csv',index=False)

except Exception as e: 
    traceback.print_exc()
    MessageBox(None,b'An error happened in permanent cell 3', b'Error', 0)
    raise ValueError("Error")


Importing network FSA_WWF_2021-11-12_5d_2021pop_S_V_Base.res1d at 2023-09-07 08:12:34.383086
Importing spilling FSA_WWF_2021-11-12_5d_2021pop_S_V_Base.ADDOUT.res1d at 2023-09-07 08:13:54.068586
Importing network FSA_WWF_2020-01-29_6d_2021pop_S_V_Base.res1d at 2023-09-07 08:14:08.220681
Importing spilling FSA_WWF_2020-01-29_6d_2021pop_S_V_Base.ADDOUT.res1d at 2023-09-07 08:15:43.423232
Importing network FSA_GA_EX-10y-24h-AES_2021p_S_V_Base.res1d at 2023-09-07 08:15:56.651824
Importing spilling FSA_GA_EX-10y-24h-AES_2021p_S_V_Base.ADDOUT.res1d at 2023-09-07 08:17:00.882423
Importing network FSA_GA_EX-5y-24h-AES_2021p_S_V_Base.res1d at 2023-09-07 08:17:11.180142
Importing spilling FSA_GA_EX-5y-24h-AES_2021p_S_V_Base.ADDOUT.res1d at 2023-09-07 08:18:14.894076
Importing network FSA_GA_EX-2y-24h-AES_2021p_S_V_Base.res1d at 2023-09-07 08:18:24.883689
Importing spilling FSA_GA_EX-2y-24h-AES_2021p_S_V_Base.ADDOUT.res1d at 2023-09-07 08:19:28.589877
Importing network FSA_DWF_2021-07-22_3d_2021po

Importing network FSA_DWF_2021-07-22_3d_2040pop_S_V_2030_Network.res1d at 2023-09-07 09:05:32.912737
Importing spilling FSA_DWF_2021-07-22_3d_2040pop_S_V_2030_Network.ADDOUT.res1d at 2023-09-07 09:06:24.312909
Importing network FSA_GA_2050M-2y-24h-AES_2040p_S_V_2030_Network.res1d at 2023-09-07 09:06:33.506189
Importing spilling FSA_GA_2050M-2y-24h-AES_2040p_S_V_2030_Network.ADDOUT.res1d at 2023-09-07 09:07:40.796663
Importing network FSA_GA_2050M-5y-24h-AES_2040p_S_V_2030_Network.res1d at 2023-09-07 09:07:53.969053
Importing spilling FSA_GA_2050M-5y-24h-AES_2040p_S_V_2030_Network.ADDOUT.res1d at 2023-09-07 09:09:01.934375
Importing network FSA_GA_2050M-10y-24h-AES_2040p_S_V_2030_Network.res1d at 2023-09-07 09:09:13.039583
Importing spilling FSA_GA_2050M-10y-24h-AES_2040p_S_V_2030_Network.ADDOUT.res1d at 2023-09-07 09:10:20.746251
Doing summation at 2023-09-07 09:10:28.580488
20th Street PS
Baynes PS
Cloverdale PS
Crescent Beach PS
East Richmond PS
Katzie PS
Langley PS
Marshend PS
New S

Sperling PS
Westridge PS #1
Westridge PS #2
White Rock PS
20th Street PS
Baynes PS
Cloverdale PS
Crescent Beach PS
East Richmond PS
Katzie PS
Langley PS
Marshend PS
New Sapperton PS
Port Coquitlam PS
Port Moody PS
Queensborough PS
Royal Avenue PS
Short Street PS
Sperling PS
Westridge PS #1
Westridge PS #2
White Rock PS
20th Street PS
Baynes PS
Cloverdale PS
Crescent Beach PS
East Richmond PS
Katzie PS
Langley PS
Marshend PS
New Sapperton PS
Port Coquitlam PS
Port Moody PS
Queensborough PS
Royal Avenue PS
Short Street PS
Sperling PS
Westridge PS #1
Westridge PS #2
White Rock PS
20th Street PS
Baynes PS
Cloverdale PS
Crescent Beach PS
East Richmond PS
Katzie PS
Langley PS
Marshend PS
New Sapperton PS
Port Coquitlam PS
Port Moody PS
Queensborough PS
Royal Avenue PS
Short Street PS
Sperling PS
Westridge PS #1
Westridge PS #2
White Rock PS
20th Street PS
Baynes PS
Cloverdale PS
Crescent Beach PS
East Richmond PS
Katzie PS
Langley PS
Marshend PS
New Sapperton PS
Port Coquitlam PS
Port Moody 

In [4]:
#PERMANENT CELL 4
MessageBox(None,b'All cells ran successfully.', b'Done', 0)

1